# Life simulation with Spark and GraphFrames 

## Настройка

Настройка на Spark

это можно делать множеством разных способов - например, так.

In [1]:
import sys
sys.path.append('/home/mk/local/spark-2.4.7-bin-hadoop2.7/python/lib/py4j-0.10.7-src.zip')
sys.path.append('/home/mk/local/spark-2.4.7-bin-hadoop2.7/python')

Подключаем Graphframes...

In [2]:
import os
os.environ["PYSPARK_SUBMIT_ARGS"] = "--packages graphframes:graphframes:0.8.1-spark2.4-s_2.11 pyspark-shell"

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f

In [4]:
spark = SparkSession.builder.master("local").appName("life_simu").getOrCreate()

In [ ]:
spark.sparkContext.setCheckpointDir('graphframes_cps')

In [5]:
from graphframes import GraphFrame
from graphframes.lib import AggregateMessages as AM

## Функции

Создадим немного функций (борьба со сложностью)....

In [6]:
def creDeltaDf():
    """ создает специальный датафрейм для ребер и соседей """
    
    return spark.createDataFrame(
        [
            [ -1, -1 ],
            [ -1, 0 ],
            [ -1, 1 ],
            [ 0, 1 ],
            [ 1, 1 ],
            [ 1, 0 ],
            [ 1, -1 ],
            [ 0, -1 ],
        ],
        ["dx", "dy"]
    )

In [116]:
def creSurrDF(df):
    """ возвращает датафрейм с колонками nv,nx,ny, содержащими координаты окружающих исходный 
    датафрейм точками
    """

    delt = creDeltaDf()
    
    res = (
        df.crossJoin(delt) # добавляем для каждой точки ее окружение
        .selectExpr("0 as nv", "x+dx as nx", "y+dy as ny") # вычисляем координаты точек окружения
        .distinct() # убираем дубликаты
        .join(      # убираем новые точки, которые попали на существующие точки
            df,(df["x"]==f.col("nx")) & (df["y"]==f.col("ny")),"left_anti"
        )
    )
    return df.union(res) # конкатенируем исходный и окружающий датафреймы

In [59]:
def creEdges(df):
    """ создает датафрейм с ребрами для графа "окружаюещего датафрейма" 
    ребра соединяют каждую ячейку с 8 соседями (граничные - только с существующими)
    """
    
    # выражения, определяющие - попала ли новая вершина в наш граф
    rx = ((f.col("x")+f.col("dx"))<=f.col("maxx"))&(f.col("minx")<=(f.col("x")+f.col("dx")))
    ry = ((f.col("y")+f.col("dy"))<=f.col("maxy"))&(f.col("miny")<=(f.col("y")+f.col("dy")))
    
    # получение id для dst вершины
    dstCol = f.concat_ws(":",(f.col("x")+f.col("dx")).cast("string"),(f.col("y")+f.col("dy")).cast("string"))
    
    # добавим к каждой строке колонки макс и мин координат
    dfn = df.crossJoin(df.selectExpr("max(x) as maxx","min(x) as minx","max(y) as maxy","min(y) as miny"))

    # создаем дуги - формируем id вершин как строку "x:y"
    delt = creDeltaDf()
    eDf = (
        dfn.crossJoin(delt)
        .withColumn("src",f.concat_ws(":",f.col("x").cast("string"),f.col("y").cast("string")))
        .withColumn("dst",f.when(rx & ry,dstCol))
        .filter("dst is not NULL")
        .select("src","dst")
    )
    return eDf

In [9]:
def creVertices(df):
    """ создает вершины по датафрейму с координатами точек 
    вершина получает id = строке "x:y"
    """
    
    vDf = (
        df
        .withColumn("id",f.concat_ws(":",f.col("x").cast("string"),f.col("y").cast("string")))
        .select("id","v")
    )
    return vDf

In [10]:
def sendMsg(gr):
    """ рассылает сообщения от живых клеток, возвращает датафрейм (id,totn) """
    res = (
        gr.aggregateMessages(
            f.sum("msg").alias("totn"),
            None,
            f.when(AM.src["v"]==1,f.lit(1)).otherwise(f.lit(0)) 
        )
    )
    return res

In [92]:
def creNewDF(df,gr):
    """ создает новый датафрейм с выжившими ячейками """

    # правила "выживания" для заполненной и пустой ячеек
    eRule = ((f.col("totn")==2)|(f.col("totn")==3))
    nRule = (f.col("totn")==3)
    
    # формирование новой ячейки по правилам выживания
    newCell = f.when(f.col("v")==1,f.when(eRule,f.lit(1)).otherwise(f.lit(0))).otherwise(f.when(nRule,f.lit(1)).otherwise(f.lit(0)))
    
    res = (
        df
        .join(gr.vertices,"id")
        .withColumn("nv", newCell)
        .filter("nv=1")
        .select("id","nv")
        .withColumn("x",f.split(f.col("id"),":").getItem(0).cast("int"))
        .withColumn("y",f.split(f.col("id"),":").getItem(1).cast("int"))
        .select("nv","x","y")
        .withColumnRenamed("nv","v")
    )
    return res

In [12]:
def printField(df):
    """ печатает поле, заданное в датафрейме """
    
    # коллектим диапазоны и координаты точек
    maxx,minx,maxy,miny = df.selectExpr("max(x)","min(x)","max(y)","min(y)").collect()[0]
    res = df.collect()

    # создаем пустое поле
    rows = []
    for i in range(maxy-miny+1):
        rows.append([None]*(maxx-minx+1))
        
    # заполняем точками (пустая=*, заполненная=-)
    for r in res:
        rows[r[2]-miny][r[1]-minx] = "*" if r[0]==1 else "-"

    # печатаем, добавляя отсутствующие в датафрейме ячейки в виде .
    for r in rows:
        pLine = []
        for el in r:
            if el is None:
                pLine.append(".")
            else:
                pLine.append(el)
        print("".join(pLine))

## Начальные данные

Несколько примеров начальных колоний клеток (для отладки и вообще)

In [50]:
# пульсар с самого начала
dfo = spark.createDataFrame(
    [
        [ 1, 0, 0 ],
        [ 1, 0, 1 ],
        [ 1, 0, 2 ],
    ],
    ["v","x","y"]
)

In [118]:
# пульсар через небольшое количесто итераций
dfo = spark.createDataFrame(
    [
        [ 1, 1, 2 ],
        [ 1, 2, 3 ],
        [ 1, 2, 2 ],
        [ 1, 2, 1 ],
        [ 1, 3, 2 ],
    ],
    ["v","x","y"]
)

## Код: его исполняем

Итерация "жизни"

In [119]:
dfo.write.format("orc").mode("overwrite").save("curdf")

Ячейку ниже нужно выполнять - каждое выполнение = очередная итерация "жизни"

In [122]:
cDf = spark.read.format("orc").load("curdf")
surDf = creSurrDF(cDf) # окружаем колонию пустыми клетками
surDf.cache()
printField(surDf)

# создаем граф (т.е. ребра и вершины)  включая пустые клетки
eDf = creEdges(surDf)
vDf = creVertices(surDf)
nGr = GraphFrame(vDf,eDf)

resDf = sendMsg(nGr) # рассылаем сообщения из "живых" клеток
nDf = creNewDF(resDf,nGr) # получаем итоговый датафрейм для следующей итерации
nDf.write.format("orc").mode("overwrite").save("curdf")

none = surDf.unpersist()

..---..
.--*--.
--*-*--
-*---*-
--*-*--
.--*--.
..---..


In [123]:
spark.stop()